In [1]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import History
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.datasets import cifar10
from keras.engine import training
from keras.layers import (Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, Activation,
                          Average, BatchNormalization, Flatten, Dense, Concatenate, LeakyReLU, Add)
from keras.losses import categorical_crossentropy
from keras.models import Model, Input
from keras.optimizers import Adam, RMSprop
from keras.utils import to_categorical
from tensorflow.python.framework.ops import Tensor
from typing import Tuple, List
import glob
import numpy as np
import pandas as pd
import os
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from scipy import ndimage

In [3]:
##################################################################################################
##################################################################################################
#데이터 로드 + augmentation
train = pd.read_csv('data/train.csv')
train_img = train.iloc[:,3:].values.reshape(-1,28,28)
test = pd.read_csv('data/test.csv')
submission = pd.read_csv('data/submission.csv')
y = train['digit']

def data_augmentation(images, labels):
    aug_images = []
    aug_labels = []    
    
    for x, y in zip(images, labels):        
        aug_images.append(x)
        aug_labels.append(y)
        
        bg_value = np.median(x)
        
        for _ in range(7):
            angle = np.random.randint(-10, 10, 1)            
            rot_img = ndimage.rotate(x, angle[0], reshape=False, cval=bg_value)
            
            shift = np.random.randint(-3, 3, 2)
            shift_img = ndimage.shift(rot_img, shift, cval=bg_value)            
            
            aug_images.append(shift_img)
            aug_labels.append(y)
            
    aug_images = np.array(aug_images)
    aug_labels = np.array(aug_labels)
    
    return aug_images, aug_labels

train_X, valid_X, train_y, valid_y = train_test_split(train_img, y, test_size = 0.2)
train_X, train_y = data_augmentation(train_X, train_y)

train_X = train_X/255.
valid_X = valid_X/255.
train_X = np.expand_dims(train_X, axis=-1)
valid_X = np.expand_dims(valid_X, axis=-1)

train_y = to_categorical(train_y, 10)
valid_y = to_categorical(valid_y, 10)

#input
input_shape = train_X.shape[1:]
model_input = Input(shape=input_shape)
##################################################################################################
##################################################################################################

In [4]:
def model_cnn_1(model_input: Tensor) -> training.Model:
    x = Conv2D(16, (3,3), padding = 'same', kernel_initializer='he_normal')(model_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(16, (3,3), padding = 'same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.4)(x)
    
    x = Conv2D(32, (3,3), padding = 'same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(32, (3,3), padding = 'same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.4)(x)
    
    x = Conv2D(64, (3,3), padding = 'same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(64, (3,3), padding = 'same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2,2))(x)
    x = Dropout(0.4)(x)
    
    x = Conv2D(128, (3,3), padding = 'same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(128, (3,3), padding = 'same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2,2))(x)
    x = Dropout(0.4)(x)
    
    x = Flatten()(x)
    x = Dense(50, kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.4)(x)
    x = Dense(10, activation='softmax')(x)
    
    model = Model(model_input, x, name='model_cnn_1')
    return model

In [5]:
def model_cnn_2(model_input: Tensor) -> training.Model:
    x = Conv2D(16, (3,3), padding = 'same', kernel_initializer='he_normal')(model_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(16, (3,3), padding = 'same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x)
    
    x = Conv2D(32, (3,3), padding = 'same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(32, (3,3), padding = 'same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x)
    
    x = Conv2D(64, (3,3), padding = 'same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(64, (3,3), padding = 'same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2,2))(x)
    x = Dropout(0.5)(x)
    
    x = Conv2D(128, (3,3), padding = 'same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(128, (3,3), padding = 'same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2,2))(x)
    x = Dropout(0.5)(x)
    
    x = Flatten()(x)
    x = Dense(50, kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(10, activation='softmax')(x)
    
    model = Model(model_input, x, name='model_cnn_2')
    return model

In [6]:
def model_cnn_3(model_input: Tensor) -> training.Model:
    x = Conv2D(16, (3,3), padding = 'same', kernel_initializer='he_normal')(model_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(16, (3,3), padding = 'same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.4)(x)
    
    x = Conv2D(32, (3,3), padding = 'same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(32, (3,3), padding = 'same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.4)(x)
    
    x = Conv2D(48, (3,3), padding = 'same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(48, (3,3), padding = 'same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2,2))(x)
    x = Dropout(0.4)(x)
    
    x = Conv2D(64, (3,3), padding = 'same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(64, (3,3), padding = 'same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2,2))(x)
    x = Dropout(0.4)(x)
    
    x = Flatten()(x)
    x = Dense(50, kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.4)(x)
    x = Dense(10, activation='softmax')(x)
    
    model = Model(model_input, x, name='model_cnn_3')
    return model

In [4]:
def model_cnn_4(model_input: Tensor) -> training.Model:
    x = Conv2D(24, (3,3), padding = 'same', kernel_initializer='he_normal')(model_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(24, (3,3), padding = 'same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.4)(x)
    
    x = Conv2D(32, (3,3), padding = 'same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(32, (3,3), padding = 'same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.4)(x)
    
    x = Conv2D(48, (3,3), padding = 'same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(48, (3,3), padding = 'same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2,2))(x)
    x = Dropout(0.4)(x)
    
    x = Conv2D(80, (3,3), padding = 'same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(80, (3,3), padding = 'same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2,2))(x)
    x = Dropout(0.4)(x)
    
    x = Flatten()(x)
    x = Dense(50, kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.4)(x)
    x = Dense(10, activation='softmax')(x)
    
    model = Model(model_input, x, name='model_cnn_4')
    return model

In [5]:
NUM_EPOCHS = 50
NUM_BATCHES = 32

#데이터 증강
datagen = ImageDataGenerator(zoom_range=0.1, #10퍼센트 확대
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             shear_range=0.1)

def compile_and_train(model: training.Model, num_epochs: int) -> Tuple [History, str]:
    model.compile(loss=categorical_crossentropy, optimizer=Adam(0.001), metrics=['accuracy'])
    
    filepath = './data/weights/' + model.name + '.ep{epoch:03d}-vl{val_loss:.5f}.hdf5'
    checkpoint = ModelCheckpoint(filepath, monitor='val_loss', mode='min', verbose=0,
                                 save_weights_only=True, save_best_only=True)
    
    tensor_board = TensorBoard(log_dir='./data/logs/', histogram_freq=0)
    
    history = model.fit(x=train_X, y=train_y,
                        epochs=NUM_EPOCHS,
                        validation_data=(valid_X,valid_y),
                        verbose=1,
                        callbacks=[checkpoint, tensor_board])
    
    weight_files = glob.glob(os.path.join(os.getcwd(), 'data/weights/*'))
    weight_file = max(weight_files, key=os.path.getctime) #most recent file
    
    return history, weight_file

In [9]:
#ensemble
conv_pool_cnn_model_1 = model_cnn_1(model_input)
_, model_cnn_1_weight_file = compile_and_train(conv_pool_cnn_model_1, NUM_EPOCHS)

conv_pool_cnn_model_2 = model_cnn_2(model_input)
_, model_cnn_2_weight_file = compile_and_train(conv_pool_cnn_model_2, NUM_EPOCHS)

conv_pool_cnn_model_3 = model_cnn_3(model_input)
_, model_cnn_3_weight_file = compile_and_train(conv_pool_cnn_model_3, NUM_EPOCHS)

conv_pool_cnn_model_4 = model_cnn_4(model_input)
_, model_cnn_4_weight_file = compile_and_train(conv_pool_cnn_model_4, NUM_EPOCHS)

Epoch 1/50
  1/410 [..............................] - ETA: 0s - loss: 2.9596 - accuracy: 0.1562WARNING:tensorflow:From C:\Users\Yubin\anaconda3\envs\tf\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
410/410 [==============================] - 7s 17ms/step - loss: 1.8773 - accuracy: 0.3431 - val_loss: 3.2538 - val_accuracy: 0.1390
Epoch 2/50
410/410 [==============================] - 6s 15ms/step - loss: 1.2592 - accuracy: 0.5788 - val_loss: 0.9134 - val_accuracy: 0.6854
Epoch 3/50
410/410 [==============================] - 6s 15ms/step - loss: 0.9708 - accuracy: 0.6761 - val_loss: 0.7191 - val_accuracy: 0.7293
Epoch 4/50
410/410 [==============================] - 6s 15ms/step - loss: 0.8148 - accuracy: 0.7252 - val_loss: 0.6638 - val_accuracy: 0.7634
Epoch 5/50
410/410 [==============================] - 

410/410 [==============================] - 5s 12ms/step - loss: 0.7063 - accuracy: 0.7660 - val_loss: 0.4104 - val_accuracy: 0.8537
Epoch 7/50
410/410 [==============================] - 5s 12ms/step - loss: 0.6438 - accuracy: 0.7842 - val_loss: 0.3472 - val_accuracy: 0.8829
Epoch 8/50
410/410 [==============================] - 5s 12ms/step - loss: 0.5920 - accuracy: 0.8032 - val_loss: 0.3799 - val_accuracy: 0.8610
Epoch 9/50
410/410 [==============================] - 5s 12ms/step - loss: 0.5392 - accuracy: 0.8216 - val_loss: 0.2991 - val_accuracy: 0.8829
Epoch 10/50
410/410 [==============================] - 5s 12ms/step - loss: 0.4969 - accuracy: 0.8320 - val_loss: 0.3293 - val_accuracy: 0.8951
Epoch 11/50
410/410 [==============================] - 5s 12ms/step - loss: 0.4801 - accuracy: 0.8402 - val_loss: 0.3883 - val_accuracy: 0.8683
Epoch 12/50
410/410 [==============================] - 5s 12ms/step - loss: 0.4531 - accuracy: 0.8454 - val_loss: 0.3548 - val_accuracy: 0.8829
Epoch 1

In [10]:
#최고 weight 값으로 모델 설정

#os.path.join(os.getcwd(), 'data/weights', 'conv_pool_cnn_model_4_pretrained_weights.hdf5')

conv_pool_cnn_model_1 = model_cnn_1(model_input)
conv_pool_cnn_model_2 = model_cnn_2(model_input)
conv_pool_cnn_model_3 = model_cnn_3(model_input)
conv_pool_cnn_model_4 = model_cnn_4(model_input)

conv_pool_cnn_model_1.load_weights(model_cnn_1_weight_file)
conv_pool_cnn_model_2.load_weights(model_cnn_2_weight_file)
conv_pool_cnn_model_3.load_weights(model_cnn_3_weight_file)
conv_pool_cnn_model_4.load_weights(model_cnn_4_weight_file)

models = [conv_pool_cnn_model_1, conv_pool_cnn_model_2, conv_pool_cnn_model_3, conv_pool_cnn_model_4]

In [10]:
def ensemble(models: List [training.Model], model_input: Tensor) -> training.Model:
    outputs = [model.outputs[0] for model in models]
    y = Average()(outputs)
    
    model = Model(model_input, y, name='ensemble')
    
    return model

def evaluate_error(model: training.Model) -> np.float64:
    pred = model.predict(valid_X, batch_size = 32)
    pred = np.argmax(pred, axis=1)
    valid_ans = np.argmax(valid_y, axis=1)
    error = np.sum(np.not_equal(pred, valid_ans)) / valid_ans.shape[0]
 
    return error

In [12]:
ensemble_model = ensemble(models, model_input)
print(evaluate_error(ensemble_model))

#test 데이터
test_X = test.drop(['id', 'letter'], axis=1).values
test_X = test_X.reshape(-1, 28, 28, 1)
test_X = test_X/255.

#결과 예측
res = ensemble_model.predict(test_X)
res = np.argmax(res, axis=1)
res = np.expand_dims(res, axis=1)

submission.digit = res
submission.to_csv('data/cvision/my_subm_ensemble_2.csv', index=False)

0.05609756097560976


In [13]:
#따로 확인용 블럭

def evaluate_error(model: training.Model) -> np.float64:
    pred = model.predict(valid_X, batch_size = 32)
    pred = np.argmax(pred, axis=1)
    valid_ans = np.argmax(valid_y, axis=1)
    error = np.sum(np.not_equal(pred, valid_ans)) / valid_ans.shape[0]
 
    return error

conv_pool_cnn_model_4 = model_cnn_4(model_input)
wfile = './data/weights/model_cnn_4.ep028-vl0.21266.hdf5'
conv_pool_cnn_model_4.load_weights(wfile)

print(evaluate_error(conv_pool_cnn_model_4))

#test 데이터
test_X = test.drop(['id', 'letter'], axis=1).values
test_X = test_X.reshape(-1, 28, 28, 1)
test_X = test_X/255.

#결과 예측
res = conv_pool_cnn_model_4.predict(test_X)
res = np.argmax(res, axis=1)
res = np.expand_dims(res, axis=1)

submission.digit = res
submission.to_csv('data/cvision/my_subm_1_mysubm_model4.csv', index=False)

0.01951219512195122
